## Win/Loss Rating Model Prediction

Load the model and make predictions

In [121]:
import pandas as pd
import numpy as np
import pymc3 as pm
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
from wl_model.spcl_case import *
plt.style.use('fivethirtyeight')
%matplotlib inline

### Get a list of all CS Games

In [122]:
import cfscrape # need nodejs
import json
scraper = cfscrape.create_scraper()
games = json.loads(scraper.get("http://thunderpick.com/api/matches").content)
games = pd.DataFrame(games['data'])
games = games[games.gameId == 6].sort_values('startTime')

bet_games = []
for i,v in games.iterrows():
    if((v['isTournament'] == False )& (v['canWager'] == True)):
        ratio = v['matchBet']['buckets'][0]['amount']/v['matchBet']['buckets'][1]['amount']
        odds = (ratio**-1+1, ratio+1)
        wr = (odds[1]/np.sum(odds)*100., odds[0]/np.sum(odds)*100.)
        det = json.loads(scraper.get('https://thunderpick.com/api/matches/'+str(v['id'])).content)['data']
        print('Date: %s  |  Event: %s  | (BO%s) %s vs. %s  |  (%.1f:%.1f) | Total Coins: %i' % 
              (v['startTime'][:10], v['championship'], det['bestOfMaps'], v['matchBet']['buckets'][0]['label'], 
               v['matchBet']['buckets'][1]['label'], wr[0], wr[1], v['matchBet']['amount']))
        bet_games.append({'1': v['matchBet']['buckets'][0]['label'], '2': v['matchBet']['buckets'][1]['label'], 'bo': det['bestOfMaps'], 'o1': odds[0], 'o2': odds[1], 'wr': wr[0]})
bet_games = pd.DataFrame(bet_games)
bet_games['1'] = bet_games['1'].str.lower()
bet_games['2'] = bet_games['2'].str.lower()

Date: 2017-11-15  |  Event: WCA 2017  | (BO3) BIG vs. Natus Vincere  |  (24.1:75.9) | Total Coins: 35960
Date: 2017-11-15  |  Event: Mother Russia  | (BO1) GoodJob vs. Tricked  |  (22.8:77.2) | Total Coins: 8774
Date: 2017-11-15  |  Event: X-BET.co Invitational  | (BO1) HAVU vs. EnVyUs Academy  |  (72.1:27.9) | Total Coins: 8346
Date: 2017-11-15  |  Event: CEVO Main Season 13  | (BO3) WASD vs. Movistar Riders  |  (61.1:38.9) | Total Coins: 5400
Date: 2017-11-15  |  Event: Mother Russia  | (BO1) GameAgents vs. Nexus  |  (41.3:58.7) | Total Coins: 5195
Date: 2017-11-15  |  Event: CEVO Main Season 13  | (BO3) Pompa Team vs. PURSI$EURA  |  (55.0:45.0) | Total Coins: 5000
Date: 2017-11-15  |  Event: Nordic Championship  | (BO1) HAVU vs. Passions  |  (60.0:40.0) | Total Coins: 5000
Date: 2017-11-15  |  Event: ECS Season 4 Development League  | (BO1) Virtus.pro vs. mousesports  |  (45.7:54.3) | Total Coins: 8938
Date: 2017-11-15  |  Event: Intel Extreme Masters 2017  | (BO1) SK vs. Astralis  

## Load Ratings Model

In [118]:
TEAM_SET = 'all_player_sc'

players = np.load('wl_model/saved_model/'+TEAM_SET+'/players.npy')
maps = np.load('wl_model/saved_model/'+TEAM_SET+'/maps.npy')
rating_model = prep_pymc_model(len(teams), len(maps))
trace = pm.backends.text.load('wl_model/saved_model/'+TEAM_SET+'/trace', model=rating_model)

h_teams = pd.read_csv('wl_model/hltv_csv/teams_w_ranking.csv').set_index('ID').dropna() # top 360 only
h_teams = fix_teams(h_teams)
h_teams.Name = h_teams.Name.str.lower()
h_matches = pd.read_csv('wl_model/hltv_csv/matchResults.csv').set_index('Match ID')
h_matches['Date'] = pd.to_datetime(h_matches['Date'])
h_players = pd.read_csv('wl_model/hltv_csv/matchLineups.csv').set_index('Match ID')
h_matches = h_matches.join(h_players)

player_col_names = ['Team 1 Player 1', 'Team 1 Player 2', 'Team 1 Player 3', 'Team 1 Player 4', 'Team 1 Player 5',
                 'Team 2 Player 1', 'Team 2 Player 2', 'Team 2 Player 3', 'Team 2 Player 4', 'Team 2 Player 5',]

In [139]:
tab_match = pd.DataFrame(np.concatenate([h_matches[['Date','Team 1 ID']+player_col_names[:5]].values, 
                h_matches[['Date', 'Team 2 ID']+player_col_names[5:]].values]), columns=['date', 'team','1','2','3','4','5']).set_index('team')
teams_from_games = h_teams[h_teams.Name.isin(np.concatenate([bet_games['1'].values, bet_games['2'].values]))].drop_duplicates()
player_table = tab_match.loc[teams_from_games.index].sort_values('date').groupby(level=0).last().dropna()

# Bet Predictions

In [141]:
money = 4500.
def sig(x):
    return 1 / (1 + np.exp(-x))
def abs_norm_interval(start,end,loc,scale):
    return (norm.cdf(end,loc,scale) - norm.cdf(start,loc,scale)) + (norm.cdf(-1*start,loc,scale) - norm.cdf(-1*end,loc,scale))

matches = bet_games[bet_games['1'].isin(teams_from_games.Name.values) & bet_games['2'].isin(teams_from_games.Name.values)]

t_rating = trace['rating']
t_map_rating = trace['rating | map']
t_alpha = 0.5
t_sigma = trace['sigma']
for i,v in matches.iterrows():
    t1_id = teams_from_games[teams_from_games.Name == v['1']].index[0]; t1_ind = np.searchsorted(players, player_table.loc[t1_id, ['1','2','3','4','5']].values);
    t2_id = teams_from_games[teams_from_games.Name == v['2']].index[0]; t2_ind = np.searchsorted(players, player_table.loc[t2_id, ['1','2','3','4','5']].values);
    trace_1 = np.sum(t_rating[:,t1_ind], axis=1); trace_2 = np.sum(t_rating[:,t2_ind], axis=1)
    mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
    diff = trace_1-trace_2
    p_wl = 1.-norm.cdf(0,loc=16*np.tanh(t_alpha*diff), scale=t_sigma)
    wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
    kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.10
    kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.10
    print('%s (%.3f) vs %s (%.3f) - P:%.2f%% - %.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
          (v['1'], mr_1, v['2'], mr_2,  v['wr']-wr_25*100, v['wr'], wr_75*100-v['wr'], kelly_pct_1*100., 
           kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))

big (0.934) vs natus vincere (0.999) - P:-15.87% - 24.06% - 29.56%  -  K: 3.2% (145) - 2.0% (90)
goodjob (0.077) vs tricked (0.584) - P:2.87% - 22.79% - 11.00%  -  K: 0.7% (32) - 5.7% (255)
havu (0.871) vs envyus academy (0.429) - P:6.38% - 72.11% - 3.83%  -  K: 4.8% (216) - 1.1% (47)
wasd (0.401) vs movistar riders (0.329) - P:14.92% - 61.11% - -0.27%  -  K: 2.3% (105) - 2.7% (121)
gameagents (-0.133) vs nexus (0.423) - P:24.27% - 41.29% - -7.39%  -  K: -0.9% (-39) - 6.4% (285)
havu (0.871) vs passions (0.770) - P:14.44% - 60.00% - 4.85%  -  K: 2.6% (115) - 2.5% (112)
virtus.pro (0.987) vs mousesports (1.248) - P:13.33% - 45.74% - -3.97%  -  K: 0.7% (31) - 4.4% (198)
sk (1.475) vs astralis (1.467) - P:20.80% - 66.00% - -10.62%  -  K: 1.6% (74) - 3.3% (150)
nip (1.255) vs cloud9 (1.323) - P:-1.44% - 39.61% - 12.83%  -  K: 2.3% (105) - 2.8% (124)
mousesports (1.248) vs virtus.pro (0.987) - P:-6.84% - 51.39% - 16.20%  -  K: 4.3% (192) - 0.8% (34)
extatus (0.666) vs gatekeepers (-0.283) -

In [ ]:
PRINT_RD_DIFF = False
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    print('---------- %s vs %s ---------------------------------' % (v['1'], v['2']))
    pred_maps = predict_map(model_played, h_matches, t1_id, t2_id)
    pred_maps = pred_maps/pred_maps.sum()
    for m,s in pred_maps.iteritems():
        m_ind = np.where(maps == m)[0][0]
        trace_1 = t_map_rating[:,m_ind,t1_ind]; trace_2 = t_map_rating[:,m_ind,t2_ind]
        mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
        diff = trace_1-trace_2
        p_wl = sig(diff)
        wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
        kappa = 32*sig(t_alpha*diff)-16.
        kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
        kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
        print('    Map: %s (%.2f)  -  %s (%.3f) vs %s (%.3f) - P:%.2f%% - %.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
             (m, s*100., v['1'], mr_1, v['2'], mr_2,  wr_25*100, v['wr'], wr_75*100, kelly_pct_1*100., 
               kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))
        
        if(PRINT_RD_DIFF):
            p_sc = [abs_norm_interval(x[0],x[1],kappa,trace['sigma'][:,m_ind]) for x in [[1.5,3.5],[3.5,5.5],[5.5,7.5],[7.5,9.5],[9.5,16]]]
            for i,sd in enumerate(['2 - 3 Rounds', '4 - 5 rounds', '6 - 7 rounds', '8 - 9 rounds', '10 rounds or more']):
                sc_25 = np.percentile(p_sc[i], 25); sc_75 = np.percentile(p_sc[i], 75)
                print('      %s : %.2f%% - %.2f%%' % (sd, sc_25*100, sc_75*100))

In [ ]:
np.inner(np.ones((8,25)), np.ones((8,25))).shape

In [ ]:
np.ones((8,25)).flatten().shape

In [ ]:
np.zeros((8000, 200))

In [ ]:
plt.ylim(0,1.2)
sns.kdeplot(trace_1, shade=True, alpha=0.65, legend=True, label=v['1'])
sns.kdeplot(trace_2, shade=True, alpha=0.65, legend=True, label=v['2'])

In [ ]:
h_bp.groupby('Match ID').first().count()

In [ ]:
h_bp